In [157]:
import pandas as pd
import numpy as np
from pymoo.optimize import minimize
import pickle

from pymoo.util.termination.f_tol import MultiObjectiveSpaceToleranceTermination
from pymoo.visualization.scatter import Scatter
import contact
from notebooks.optimization_problems.constraints import Requirements
from pyreport import PlotUtil
import matplotlib.pyplot as plt

%matplotlib

plt.close('all')

Using matplotlib backend: Qt5Agg


In [158]:
columns = ['strand_name', 'tof',
           'r_a_x', 'r_a_y', 'r_a_z',
           'v_a_x', 'v_a_y', 'v_a_z',
           'r_b_x', 'r_b_y', 'r_b_z',
           'd',
           'r_ab_sff_x', 'r_ab_sff_y', 'r_ab_sff_z']

file_path = "raw/eo_telesat_post_processed.h5"

row_limit = -1

store_500km = pd.HDFStore(file_path)

instances_500km_df = store_500km.select('contact_instances', 'columns = %s' % str(columns), stop = row_limit)

# Sort by pass id
instances_500km_df = instances_500km_df.sort_index(0)

In [159]:
def get_selection(problem, res):
    x_pass = res.X[:, problem.x_indices['pass']].astype('bool')
    for i in range(x_pass.shape[0]):
        x_pass[i, :] = contact.down_select_passes(x_pass[i, :], problem.O_matrix)

    # Transmit powers
    x_Ptx_dBm = res.X[:, problem.x_indices['power']].astype('float64')
    x_Ptx_dBm[~x_pass] = np.NaN
    #x_Ptx_dBm = np.nanmax(x_Ptx_dBm, axis=1)

    # Antenna gain
    x_Gtx_dBi = np.squeeze(res.X[:, problem.x_indices['antenna']].astype('float64'))

    return x_pass, x_Ptx_dBm, x_Gtx_dBi

def recompute_obj(problem, res, scale_factors = np.array([1/-1e9, 1, 1/1e3, 1])):
    # Recompute all four objectives
    f_pointing = np.empty_like(res.F[:,0])
    f_energy = np.empty_like(res.F[:,0])
    f_latency = np.empty_like(res.F[:,0])
    f_throughput = np.empty_like(res.F[:,0])

    for i in range(len(f_throughput)):
        ff, gg = problem.evaluate_unmasked(res.X[i,:])
        ff = ff * scale_factors
        f_pointing[i] = ff[3] # s
        f_energy[i] = ff[2] # Kilo Joule
        f_latency[i] = ff[1]   # s
        f_throughput[i] = ff[0]   # Gigabit

    return f_throughput, f_latency, f_energy, f_pointing

In [160]:
plt.close('all')
def plot_performance(axs, problem, setting, res, case, target, scale_factors = np.array([1/-1e9, 1, 1/1e3, 1]) ):

    x_pass, x_Ptx_dBm, x_Gtx_dBi = get_selection(problem, res)
    f_throughput, f_latency, f_energy, f_pointing = recompute_obj(problem, res, scale_factors)

    # Plotting
    #axs = axs.flatten()

    ax = axs[0]
    ax.grid(True)
    ax.scatter(f_energy, f_throughput, marker='.', s=1)
    ax.set_xlabel("Energy used [kJ / orbit]")
    ax.set_ylabel("Throughput [GB / orbit]")
    ax.set_axisbelow(True)

    ax = axs[1]
    ax.grid(True)
    ax.scatter(f_pointing, f_throughput, marker='.', s=1)
    ax.set_xlabel("Pointing duty cycle [%]")
    ax.set_ylabel("Throughput [GB / orbit]")
    ax.set_axisbelow(True)

    ax = axs[2]
    ax.grid(True)
    ax.scatter(f_energy, f_pointing, marker='.', s=1)
    ax.set_xlabel("Energy used [kJ / orbit]")
    ax.set_ylabel("Pointing duty cycle [%]")
    ax.set_axisbelow(True)

    # ax = axs[3]
    # ax.grid(True)
    # ax.scatter(f_energy, f_latency, marker='.', s=1)
    # ax.set_xlabel("Energy used [kJ / orbit]")
    # ax.set_ylabel("Avg. Latency [s]")
    # ax.set_axisbelow(True)
    #
    # ax = axs[4]
    # ax.grid(True)
    # ax.scatter(f_pointing, f_latency, marker='.', s=1)
    # ax.set_xlabel("Pointing duty cycle [%]")
    # ax.set_ylabel("Avg. Latency [s]")
    # ax.set_axisbelow(True)
    #
    # ax = axs[5]
    # ax.grid(True)
    # ax.scatter(f_latency, f_throughput, marker='.', s=1)
    # ax.set_xlabel("Avg. Latency [s]")
    # ax.set_ylabel("Throughput [GB / orbit]")
    # ax.set_axisbelow(True)
    #
    # axs[6].set_axis_off()
    # axs[7].set_axis_off()

    axs[3].set_axis_off()
    axs[4].set_axis_off()
    ax = axs[5]
    ax.legend(fontsize=8)
    ax.set_axis_off()

    plt.tight_layout()

# Plot the optimizations
fig = plt.figure(figsize=(3.2*3, 2.4*2))
axs = fig.subplots(2,3, gridspec_kw={'height_ratios': [1, 1]})
axs = axs.flatten()

case, target = pickle.load(open('raw/eo_settings.pkl', 'rb'))

N_orbits = case['T_sim_s'] / case['T_orbit_s']
scale_factors = np.array([1/-1e9, 1, 1/1e3, 1]) * \
                np.array([1/N_orbits, 1, 1/N_orbits, 1/case['T_sim_s']]) * \
                np.array([1/8, 1, 1, 100])

problem, setting, res = pickle.load(open('raw/eo_Telesat_unsga3_100_25.pkl', 'rb'))
plot_performance(axs, problem, setting, res, case, target, scale_factors)
axs[5].scatter(np.NaN, np.NaN, marker='.', s=1, label='NSGA-3')

# problem, setting, res = pickle.load(open('raw/eo_Telesat_rnsga2_e0_01_100_25.pkl', 'rb'))
# plot_performance(axs, problem, setting, res, case, target, scale_factors)
# axs[5].scatter(np.NaN, np.NaN, marker='.', s=1, label='R-NSGA-2, $\epsilon$ = 0.01')
#
# problem, setting, res = pickle.load(open('raw/eo_Telesat_rnsga2_e0_001_200_50.pkl', 'rb'))
# plot_performance(axs, problem, setting, res, case, target, scale_factors)
# axs[5].scatter(np.NaN, np.NaN, marker='.', s=1, label='R-NSGA-2, $\epsilon$ = 0.001')
#
# problem, setting, res = pickle.load(open('raw/eo_Telesat_rnsga2_e0_0001_200_50.pkl', 'rb'))
# plot_performance(axs, problem, setting, res, case, target, scale_factors)
# axs[5].scatter(np.NaN, np.NaN, marker='.', s=1, label='R-NSGA-2, $\epsilon$ = 1e-4')

problem, setting, res = pickle.load(open('raw/eo_Telesat_rnsga2_e1e-9_200_50.pkl', 'rb'))
plot_performance(axs, problem, setting, res, case, target, scale_factors)
axs[5].scatter(np.NaN, np.NaN, marker='.', s=1, label='R-NSGA-2, $\epsilon$ = 1e-9')

No handles with labels found to put in legend.


In [161]:
# Set limits
#axs[0].set_xlim(np.array([axs[0].get_xlim()[0], case['E_Jorbit_max'] * N_orbit * scale_factors[2]  ]) )
#axs[0].set_ylim(np.array([case['T_bitorbit_min'], case['T_bitorbit_max']]) * -1 * N_orbit * scale_factors[0])
axs[1].set_xlim([0, 100])
# axs[1].set_ylim([])
# axs[2].set_xlim([])
axs[2].set_ylim([0, 100])

points = []

# Targets
p = np.array([case['T_bitorbit_target'], case['L_sorbit_target'], case['E_Jorbit_target'], case['P_sorbit_target']/3])
p = p * np.array([-1, 1, 1, 1]) * np.array([N_orbits, 1, N_orbits, N_orbits])
p_fmt = p * scale_factors
points.append({'ind': -1, 'p': p, 'args': [], 'kwargs': {'s': 10, 'color': 'r', 'marker': 'x', 'label': 'Target\n(T=%d GB/orbit, L=%d s, E=%d kJ, P=%0.2f %%)' % tuple(p_fmt)}})

problem, setting, res = pickle.load(open('raw/eo_Telesat_rnsga2_e0_001_100_25.pkl', 'rb'))
p = np.zeros(4)
p[[0,2,3]] = res.algorithm.survival.ref_points
p_fmt = p * scale_factors
points.append({'ind': -1, 'p': p, 'args': [], 'kwargs': {'s': 10, 'color': 'g', 'marker': 'x', 'label': 'Target\n(T=%d GB/orbit, L=%d s, E=%d kJ, P=%0.2f %%)' % tuple(p_fmt)}})

problem, setting, res = pickle.load(open('raw/eo_Telesat_unsga3_100_25.pkl', 'rb'))

# # Graphically closest to target
# ref = np.array([case['T_bitorbit_target'] * -1, case['E_Jorbit_target']]) * N_orbits
# diff_graph = np.abs(res.F[:, [0, 2]] - ref) * scale_factors[[0,2]] / np.array([np.diff(axs[0].get_ylim())[0], np.diff(axs[0].get_xlim())[0]])
# ind = np.argmin(np.linalg.norm(diff_graph, axis=1))
# p = np.array(recompute_obj(problem, res, scale_factors=1))
# p = p[:, ind]
# p_fmt = p * scale_factors
# points.append({'ind': ind, 'p': p, 'args': [], 'kwargs': {'s': 10, 'color': 'tab:red', 'marker': '+', 'label': 'Target\n(T=%d GB/orbit, L=%d s, E=%d kJ, P=%0.2f %%)' % tuple(p_fmt)}})

# Closest to target
ref = np.array([case['T_bitorbit_target'] * -1, case['E_Jorbit_target']]) * N_orbits
diff= np.abs(res.F[:, [0, 2]] - ref)
ind_close = np.argmin(np.linalg.norm(diff, axis=1))
p = np.array(recompute_obj(problem, res, scale_factors=1))
p = p[:, ind_close]
p_fmt = p * scale_factors
points.append({'ind': ind_close, 'p': p, 'args': [], 'kwargs': {'s': 10, 'color': 'tab:red', 'marker': '+', 'label': 'Target\n(T=%d GB/orbit, L=%d s, E=%d kJ, P=%0.2f %%)' % tuple(p_fmt)}})

# # Closest to throughput
# ref = case['T_bitorbit_target'] * -1 * N_orbits
# ind = np.argmin(np.abs(res.F[:, 0] - ref))
# p = np.array(recompute_obj(problem, res, scale_factors=1))
# p = p[:, ind]
# p_fmt = p * scale_factors
# points.append({'ind': ind, 'p': p, 'args': [], 'kwargs': {'s': 10, 'color': 'tab:red', 'marker': '+', 'label': 'Target\n(T=%d GB/orbit, L=%d s, E=%d kJ, P=%0.2f %%)' % tuple(p_fmt)}})
#
# # Closest to energy
# ref = case['E_Jorbit_target'] * N_orbits
# ind = np.argmin(np.abs(res.F[:, 2] - ref))
# p = np.array(recompute_obj(problem, res, scale_factors=1))
# p = p[:, ind]
# p_fmt = p * scale_factors
# points.append({'ind': ind, 'p': p, 'args': [], 'kwargs': {'s': 10, 'color': 'tab:red', 'marker': '+', 'label': 'Target\n(T=%d GB/orbit, L=%d s, E=%d kJ, P=%0.2f %%)' % tuple(p_fmt)}})

for point in points:
    p = point['p'] * scale_factors
    axs[0].scatter(p[2], p[0], *point['args'], **point['kwargs'])
    axs[1].scatter(p[3], p[0], *point['args'], **point['kwargs'])
    axs[2].scatter(p[2], p[3], *point['args'], **point['kwargs'])
    axs[5].scatter(np.NaN, np.NaN, *point['args'], **point['kwargs'])

axs[5].legend(fontsize=8)

plt.savefig('D:/git/thesis_report_ae/figures/relay_optimization/eo_telesat_result.pdf')
#plt.savefig('D:/git/thesis_report_ae/figures/relay_optimization/eo_telesat_range_selection_%s_%d.svg' % (setting['label'], res.algorithm.n_gen))

In [162]:
problem, setting, res = pickle.load(open('raw/eo_Telesat_unsga3_100_25.pkl', 'rb'))

x_pass, x_Ptx_dBm, x_Gtx_dBi = get_selection(problem, res)
f_throughput, f_latency, f_energy, f_pointing = recompute_obj(problem, res, scale_factors)

fig = plt.figure(figsize=(3.2*3, 2.4*3))
ax = fig.subplots()

ax.scatter(x_Gtx_dBi, f_throughput)


In [163]:
pass_ind = np.nonzero(x_pass[ind_close,:])[0]

T_orbit = case['T_orbit_s']
T_sim = case['T_sim_s']

fig = plt.figure(figsize=(3.2, 2.4))

# Plot all passes
for i, pass_df in instances_500km_df.groupby(level=0):
    p = pass_df.index[0][0] - 1 # Pass index
    tof = pass_df.tof.values
    d = pass_df.d.values/1000
    line, = plt.plot(tof, d, linewidth=0.1, color='tab:grey')
    #plt.text(tof[0], d[0], '%d' % p)

# Plot used passes
for i, pass_df in instances_500km_df.groupby(level=0):
    p = pass_df.index[0][0] - 1 # Pass index
    tof = pass_df.tof
    d = pass_df.d/1000
    if p in pass_ind:
        line, = plt.plot(tof, d, linewidth=0.5, color='tab:red')


plt.xlabel('Time of flight [s]')
plt.ylabel('Range [km]')
plt.xlim((0, T_sim))

plt.grid()
PlotUtil.apply_report_formatting()
fig.set_size_inches(3.2*2, 2.4, forward=True)

plt.tight_layout()
plt.savefig('D:/git/thesis_report_ae/figures/relay_optimization/eo_telesat_range_selection.pdf')
plt.savefig('D:/git/thesis_report_ae/figures/relay_optimization/eo_telesat_range_selection_%s_%d.svg' % (setting['label'], res.algorithm.n_gen))

In [164]:
store_500km.close()